In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from time import time

In [23]:
import warnings
warnings.filterwarnings('ignore')

### Organize data

In [2]:
df=pd.read_excel('raw_data_bbg.xlsx',
                 sheet_name='px_data',
                header=0)
df['Date']=pd.to_datetime(df['Date'])

# Because of saving issues, 39 NA appeared at the tail
df = df.iloc[:-39]

In [3]:
px=df.dropna(axis=1) #drop stocks without full data, 88 stocks remain
px=px.set_index('Date')
#px

In [4]:
stock_list=px.columns
stock_list[:5] #show first 5

Index(['AAPL UW Equity', 'ABT UN Equity', 'ACN UN Equity', 'ADBE UW Equity',
       'AIG UN Equity'],
      dtype='object')

In [26]:
len(stock_list)

88

In [5]:
#these will be part of the feature space
ma11=px.rolling(11).mean() 
ma50=px.rolling(50).mean()
ma200=px.rolling(200).mean()

In [6]:
df2=pd.read_excel('raw_data_bbg.xlsx',sheet_name='earn_data',header=0)

In [7]:

earn_dates=df2.iloc[:, range(2,807,8) ]
earn_dates.columns=df.columns[1:]
#earn_dates  #holds the earnings announcement dates

In [8]:
EPS=df2.iloc[:, range(5,807,8) ]
EPS.columns=df.columns[1:]
#EPS   #holds the 'comparable' EPS, i.e. not basic EPS

In [9]:
est_EPS=df2.iloc[:, range(6,807,8) ]
est_EPS.columns=df.columns[1:]
#est_EPS   #holds the forecast consensus for the comparable EPS

### Implementation of CorrelNowcast algorithm from paper

In [10]:
from sklearn.linear_model import Ridge
def my_CorrelNowcast(s, W, nu, X_all, y_all, Info, start_date, end_date): 
    # s for bbg ticker, W for window size, nu for regularization param
    # output inclusive of end date
    
    begin = np.where(X_all.index==start_date)[0][0] #get integer index corresponding to date
    end = np.where(X_all.index==end_date)[0][0] 

    Ew_X = X_all.iloc[begin-W:begin,:] 
    Ew_y = y_all.iloc[begin-W:begin,0].to_numpy() #0 for the EPS column

    fit = Ridge(alpha=nu).fit(Ew_X, Ew_y) #initialize

    P = None
    Q = None
    output = np.zeros(end-begin+1)
    for t in range(begin, end+1):

        P = pd.DataFrame([X_all.iloc[t,:].to_numpy()]) \
            if P is None else P.append(pd.DataFrame([X_all.iloc[t,:].to_numpy()]))

        if Q is None:
            Q = list(fit.predict(X_all.iloc[t,:].to_numpy().reshape(1,-1)))
        else:
            Q.append(fit.predict(X_all.iloc[t,:].to_numpy().reshape(1,-1)))
        
        output[t-begin] = np.mean(Q)

        if X_all.index[t] in Info.index: #if we are at an earnings announcement date
            P.columns=Ew_X.columns #needed for correct appending
            Ew_X=Ew_X.append(P) #works as the for loop in the paper
            Ew_y=np.concatenate([Ew_y, [Info.loc[X_all.index[t], 'EPS']]*len(Q)])
            P=None
            Q=None
            
            if Ew_X.shape[0]>W:
                Ew_X=Ew_X.iloc[-W:,:] #keep only W rows
                Ew_y=Ew_y[-W:]
            
            fit=Ridge(alpha=nu).fit(Ew_X, Ew_y) #retrain
        
    return output

### Cross validation for nu and W (this code takes 6 hrs to run, results saved as text below) 

In [31]:
from sklearn.metrics import mean_absolute_percentage_error # need sklearn 0.24

nu=np.array([10, 100, 1000, 3000, 5000, 10000])
W=np.array([11, 50, 125, 200, 250, 350]) #not doing 500 and 700

nu = [10]
W = [11]

cv_result=dict()


times = {}


for a in nu:
    for b in W:
        print('Now in loop: nu={:d}, W={:d}...'.format(a,b), end='')
        MRE = np.zeros(len(stock_list))
        for i, ticker in enumerate(stock_list):
            start = time()
            print(i, end='') #visualize progress
            cor11=px[ticker].rolling(11).corr(px)
            cor50=px[ticker].rolling(50).corr(px)
            cor200=px[ticker].rolling(200).corr(px)
            

            temp1=pd.merge(px,cor11*ma11, left_index=True, right_index=True, suffixes=['_px', '_11'])
            temp2=pd.merge(temp1,cor50*ma50, left_index=True, right_index=True, suffixes=[None, None] )
            X=pd.merge(temp2,cor200*ma200, left_index=True, right_index=True, suffixes=['_50', '_200'])
            X=X.dropna() #some top rows have Nan due to rolling averge calculations

            Earnings_info=pd.DataFrame({'Announcement_dt':earn_dates[ticker].dropna(),
                                  'EPS':EPS[ticker].dropna(),
                                  'est_EPS':est_EPS[ticker].dropna()})
            Earnings_info=Earnings_info.set_index('Announcement_dt')

            #X and y_extra_info shall hv same no. of rows
            y_extra_info=pd.merge(pd.DataFrame(index=X.index),Earnings_info, left_index=True, right_index=True, how='left')
            y_extra_info=y_extra_info.fillna(method='bfill') #fill NaN with next earnings data, there will be some Nan
                                                #remaining at the end since earnings not out yet. We
                                                #won't be using those rows
            
            
            #cross validation period
            start_date='2013-02-08'
            end_date='2015-02-09'
            
            start = time()

            y_pred=my_CorrelNowcast(ticker, b, a, X, y_extra_info, Earnings_info, start_date, end_date)
            MRE[i]=mean_absolute_percentage_error(y_extra_info.loc[start_date:end_date,'EPS'], y_pred)
            cv_result[(a,b)]=np.mean(MRE)
            
            end = time()
            
            times[ticker] = end - start
        print('')   
            

cv_result

Now in loop: nu=10, W=11...0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687


{(10, 11): 0.21895543935318215}

 cv_result saved as text here:{(10, 11): 0.2187638191270434,
 (10, 50): 0.22906451942601178,
 (10, 125): 0.31202466937432827,
 (10, 200): 0.3825881964232682,
 (10, 250): 0.33047717422058687,
 (10, 350): 0.3705547594743095,
 (100, 11): 0.21851150849725237,
 (100, 50): 0.22558408007391287,
 (100, 125): 0.26659903572682564,
 (100, 200): 0.2640797603053457,
 (100, 250): 0.2703969553958662,
 (100, 350): 0.29776815242467924,
 (1000, 11): 0.21763164046085368,
 (1000, 50): 0.22069437118027563,
 (1000, 125): 0.23746854866082315,
 (1000, 200): 0.22573256964428604,
 (1000, 250): 0.23435294423273242,
 (1000, 350): 0.25400878483123424,
 (3000, 11): 0.21708701575300682,
 (3000, 50): 0.21948953352799538,
 (3000, 125): 0.22872920667907815,
 (3000, 200): 0.2225562088940296,
 (3000, 250): 0.2264492982777717,
 (3000, 350): 0.24119754914558456,
 (5000, 11): 0.21686743840521164,
 (5000, 50): 0.21898640741750486,
 (5000, 125): 0.22568012060609888,
 (5000, 200): 0.22240374315178094,
 (5000, 250): 0.2248758424106235,
 (5000, 350): 0.23697964201516844,
 (10000, 11): 0.21669252019829163,
 (10000, 50): 0.21852854901945928,
 (10000, 125): 0.2232050513633187,
 (10000, 200): 0.22266263741956352,
 (10000, 250): 0.2246121264495324,
 (10000, 350): 0.2330335124678897}

In [32]:
times

{'AAPL UW Equity': 5.571969747543335,
 'ABT UN Equity': 5.601913928985596,
 'ACN UN Equity': 5.611130952835083,
 'ADBE UW Equity': 5.629430532455444,
 'AIG UN Equity': 5.502375602722168,
 'ALL UN Equity': 5.471943378448486,
 'AMGN UW Equity': 5.467813014984131,
 'AMT UN Equity': 5.590137481689453,
 'AMZN UW Equity': 5.553435325622559,
 'AXP UN Equity': 5.501429080963135,
 'BA UN Equity': 5.5796308517456055,
 'BAC UN Equity': 5.549272298812866,
 'BIIB UW Equity': 5.53131628036499,
 'BK UN Equity': 5.545590877532959,
 'BKNG UW Equity': 5.399063587188721,
 'BLK UN Equity': 5.3539674282073975,
 'BMY UN Equity': 5.385997533798218,
 'BRK/B UN Equity': 5.358097314834595,
 'C UN Equity': 5.360106706619263,
 'CAT UN Equity': 5.298303127288818,
 'CHTR UW Equity': 5.346423149108887,
 'CL UN Equity': 5.327218532562256,
 'CMCSA UW Equity': 5.345793724060059,
 'COF UN Equity': 5.343923330307007,
 'COP UN Equity': 5.2739245891571045,
 'COST UW Equity': 5.323768138885498,
 'CRM UN Equity': 5.317119598

In [ ]:
min(cv_result, key=cv_result.get) #run this if you've run the cv

Best parameter combo with lowest MRE in cv: nu=10000, W=11

### Run model on test set

In [13]:
from sklearn.metrics import mean_absolute_percentage_error # need sklearn 0.24

MRE_model=np.zeros(len(stock_list))
MRE_bbg_est=np.zeros(len(stock_list))
print('Index running...')
for i, ticker in enumerate(stock_list):
    print(i,'',end='') #visualize progress
    cor11=px[ticker].rolling(11).corr(px)
    cor50=px[ticker].rolling(50).corr(px)
    cor200=px[ticker].rolling(200).corr(px)

    temp1=pd.merge(px,cor11*ma11, left_index=True, right_index=True, suffixes=['_px', '_11'])
    temp2=pd.merge(temp1,cor50*ma50, left_index=True, right_index=True, suffixes=[None, None] )
    X=pd.merge(temp2,cor200*ma200, left_index=True, right_index=True , suffixes=['_50', '_200'])
    X=X.dropna() #some top rows have Nan due to rolling averge calculations

    Earnings_info=pd.DataFrame({'Announcement_dt':earn_dates[ticker].dropna(),
                          'EPS':EPS[ticker].dropna(),
                          'est_EPS':est_EPS[ticker].dropna()})
    Earnings_info=Earnings_info.set_index('Announcement_dt')

    #X and y_extra_info shall hv same no. of rows
    y_extra_info=pd.merge(pd.DataFrame(index=X.index),Earnings_info, left_index=True, right_index=True, how='left')
    y_extra_info=y_extra_info.fillna(method='bfill') #fill NaN with next earnings data, there will be some Nan
                                        #remaining at the end since earnings not out yet. We
                                        #won't be using those rows
    #test data period - 4years
    start_date='2015-03-09' #start date at least 11 trading days after cv period
    end_date='2019-03-11'

    y_pred=my_CorrelNowcast(ticker, 11, 10000, X, y_extra_info, Earnings_info,start_date,end_date)
    MRE_model[i]=mean_absolute_percentage_error(y_extra_info.loc[start_date:end_date,'EPS'], y_pred)
    MRE_bbg_est[i]=mean_absolute_percentage_error(y_extra_info.loc[start_date:end_date,'EPS'],
                                                  y_extra_info.loc[start_date:end_date,'est_EPS'])
    

print('\nTest period: 2015-03-09 to 2019-03-11')
print('MRE from model:', np.mean(MRE_model))
print('MRE from analyst estimates:', np.mean(MRE_bbg_est))


Index running...
0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 
Test period: 2015-03-09 to 2019-03-11
MRE from model: 0.23691950516088855
MRE from analyst estimates: 0.11550951150632237


MRE from model actually higher then MRE from analyst estimates. This does not support the paper's claim.